## Conversion to fif: BJH017

This script takes the raw, original data file we recieved from WashU and converts it to a fif file with the photodiode as a new stim channel


Just need to change fields in `prep_paths`

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import mat73
import re
from neurodsp.timefrequency import compute_wavelet_transform
from BCI2kReader import BCI2kReader as b2k
import os
import mne
from tabulate import tabulate
import IPython
import seaborn as sns

In [2]:
## prep paths

subject = 'BJH017'
orig_data_fi = '/home/brooke/knight_server/remote/WashU/data/PacmanTask/BJH017/PacmanTask/ECOG001/ECOGS001R01.dat'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}/ieeg"

In [4]:
## load data

with b2k.BCI2kReader(orig_data_fi) as data: #opens a stream to the dat file
    my_signals = data.signals
    my_states = data.states
    my_params = data.parameters

In [10]:
## check structure if necessary

my_states
my_params

{'SourceCh': 272,
 'SampleBlockSize': 200,
 'SamplingRate': 2000,
 'ChannelNames': ['EMPTY',
  'EMPTY_2',
  'EMPTY_3',
  'EMPTY_4',
  'REF1',
  'REF2',
  'AR1',
  'AR2',
  'AR3',
  'AR4',
  'AR5',
  'AR6',
  'AR7',
  'AR8',
  'AR9',
  'AR10',
  'AR11',
  'AR12',
  'AR13',
  'AR14',
  'BR1',
  'BR2',
  'BR3',
  'BR4',
  'BR5',
  'BR6',
  'BR7',
  'BR8',
  'BR9',
  'BR10',
  'BR11',
  'BR12',
  'BR13',
  'BR14',
  'BR15',
  'BR16',
  'CR1',
  'CR2',
  'CR3',
  'CR4',
  'CR5',
  'CR6',
  'CR7',
  'CR8',
  'CR9',
  'CR10',
  'CR11',
  'CR12',
  'CR13',
  'CR14',
  'CR15',
  'CR16',
  'DR1',
  'DR2',
  'DR3',
  'DR4',
  'DR5',
  'DR6',
  'DR7',
  'DR8',
  'DR9',
  'DR10',
  'DR11',
  'DR12',
  'DR13',
  'DR14',
  'ER1',
  'ER2',
  'ER3',
  'ER4',
  'ER5',
  'ER6',
  'ER7',
  'ER8',
  'ER9',
  'ER10',
  'ER11',
  'ER12',
  'FR1',
  'FR2',
  'FR3',
  'FR4',
  'FR5',
  'FR6',
  'FR7',
  'FR8',
  'FR9',
  'FR10',
  'FR11',
  'FR12',
  'GR1',
  'GR2',
  'GR3',
  'GR4',
  'GR5',
  'GR6',
  'GR7',

In [6]:
## create info field 

info = mne.create_info(ch_names = my_params['ChannelNames'],
                       sfreq = my_params['SamplingRate'],
                       ch_types = 'seeg')

info

<Info | 7 non-empty values
 bads: []
 ch_names: EMPTY, EMPTY_2, EMPTY_3, EMPTY_4, REF1, REF2, AR1, AR2, AR3, ...
 chs: 272 sEEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 1000.0 Hz
 meas_date: unspecified
 nchan: 272
 projs: []
 sfreq: 2000.0 Hz
>

In [7]:
## create mne object
mne_df = mne.io.RawArray(my_signals, info)

Creating RawArray with float64 data, n_channels=272, n_times=3739200
    Range : 0 ... 3739199 =      0.000 ...  1869.600 secs
Ready.


In [12]:
len(my_states['Trial_on_off'][0])

3739200

In [13]:
my_signals.shape

(272, 3739200)

In [8]:
## create stim channel and add it to mne object

info = mne.create_info(['STI'], my_params['SamplingRate'], ['stim'])
stim_raw = mne.io.RawArray( my_states['Trial_on_off'], info)
mne_df.add_channels([stim_raw], force_update_info=True)

Creating RawArray with float64 data, n_channels=1, n_times=3739200
    Range : 0 ... 3739199 =      0.000 ...  1869.600 secs
Ready.


<RawArray | 273 x 3739200 (1869.6 s), ~7.61 GB, data loaded>

In [9]:
## save it

mne_df.save(f"{raw_data_dir}/{subject}_raw_ieeg.fif", overwrite = False)

Writing /home/brooke/pacman/raw_data/BJH017/ieeg/BJH017_raw_ieeg.fif
Overwriting existing file.
Writing /home/brooke/pacman/raw_data/BJH017/ieeg/BJH017_raw_ieeg-1.fif
Closing /home/brooke/pacman/raw_data/BJH017/ieeg/BJH017_raw_ieeg-1.fif
Closing /home/brooke/pacman/raw_data/BJH017/ieeg/BJH017_raw_ieeg.fif
[done]
